```{contents}
```
## State Reducers 

In LangGraph, a **State Reducer** is a deterministic function that defines **how multiple state updates are merged into the global state**.
Reducers make concurrent execution, partial updates, and cyclic workflows **safe, predictable, and scalable**.

---

### **1. Motivation and Intuition**

In real workflows:

* Multiple nodes may update the **same field**
* Nodes may run **in parallel**
* Cycles repeatedly modify the same variables

Without reducers, state updates would be ambiguous and error-prone.

Reducers provide **formal conflict resolution**.

---

### **2. Conceptual Model**

```
Node A update ─┐
               ├─> Reducer ──> Global State
Node B update ─┘
```

The reducer defines **how** those updates are combined.

---

### **3. Why Reducers Exist**

| Problem            | Without Reducer | With Reducer             |
| ------------------ | --------------- | ------------------------ |
| Concurrent writes  | Overwrite       | Merged deterministically |
| Partial updates    | Lost data       | Safely accumulated       |
| Cyclic updates     | Corruption      | Stable evolution         |
| Multi-agent memory | Race conditions | Coordinated memory       |

---

### **4. Formal Definition**

A reducer is a function:

```python
def reducer(old_value, new_value) -> merged_value
```

It is registered per state field.

---

### **5. Declaring State with Reducers**

```python
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph

def append_messages(old, new):
    return old + new

class State(TypedDict):
    messages: Annotated[list, append_messages]
    score: int
```

Here:

* `messages` uses a reducer
* `score` uses default overwrite behavior

---

### **6. How Reducers Work at Runtime**

Suppose two nodes both return:

```python
{"messages": ["A"]}
{"messages": ["B"]}
```

Reducer execution:

```python
append_messages(["existing"], ["A"]) → ["existing", "A"]
append_messages(["existing", "A"], ["B"]) → ["existing", "A", "B"]
```

Final state:

```
["existing", "A", "B"]
```

---

### **7. Common Reducer Patterns**

| Reducer Pattern   | Purpose             |
| ----------------- | ------------------- |
| Append            | Logs, conversations |
| Union             | Deduplicate items   |
| Max / Min         | Track best score    |
| Sum               | Accumulate counters |
| Merge dicts       | Structured memory   |
| Replace-if-better | Optimization        |

---

### **8. Practical Example**

```python
def add_scores(old, new):
    return old + new

class State(TypedDict):
    scores: Annotated[list, add_scores]
    total: int
```

Node updates:

```python
return {"scores": [5], "total": 5}
```

Global result:

```
scores = [1, 2, 5]
total  = 5
```

`scores` accumulated safely, `total` overwritten.

---

### **9. Reducers in Cycles & Agents**

Reducers enable:

* **Long-running agents**
* **Memory accumulation**
* **Self-improving loops**

Without reducers, cyclic systems quickly corrupt state.

---

### **10. Anti-Patterns to Avoid**

| Anti-Pattern               | Why Dangerous       |
| -------------------------- | ------------------- |
| Blind overwrite            | Loses information   |
| Storing raw LLM outputs    | Explodes memory     |
| Reducers with side effects | Break determinism   |
| Non-idempotent reducers    | Inconsistent replay |

---

### **11. Mental Model**

> **Reducers = database transaction rules**
> **State = shared ledger**

---

### **12. When to Use Reducers**

Use reducers for:

* Messages
* Logs
* Plans
* Metrics
* Agent memory
* Any accumulating field

---

### **13. Key Design Principle**

> **Every state field with multiple writers must have a reducer.**

This single rule enables **correct concurrency, safe cycles, and scalable LangGraph systems**.

---

### **Demonstration**



In [1]:
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END

def append(old, new):
    return old + new

class State(TypedDict):
    messages: Annotated[list, append]

def node_a(state):
    return {"messages": ["hello"]}

def node_b(state):
    return {"messages": ["world"]}

builder = StateGraph(State)
builder.add_node("A", node_a)
builder.add_node("B", node_b)

builder.set_entry_point("A")
builder.add_edge("A", "B")
builder.add_edge("B", END)

graph = builder.compile()

print(graph.invoke({"messages": []}))


{'messages': ['hello', 'world']}




**Output**

```
{'messages': ['hello', 'world']}
```

Both updates merged **deterministically** through the reducer.
